In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import pandas as pd

In [ ]:
# Define dataset directory
dataset_dir = "dataset"

# Get all images and their labels
image_paths = []
labels = []

image_names = os.listdir(dataset_dir)
class_names = set(['_'.join(img.split('_')[:2]) for img in image_names])
num_classes = len(class_names)

for img_name in image_names:
    image_paths.append(os.path.join(dataset_dir, img_name))
    label = '_'.join(img_name.split('_')[:2])
    labels.append(label)  # The folder name is the label

# Create a DataFrame
df = pd.DataFrame({"image_path": image_paths, "label": labels})

# Split into train (80%) and validation (20%)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}, Classes : {num_classes}")

In [ ]:
def build_improved_model(input_shape=(64, 64, 3), num_classes=num_classes):
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), activation="relu", input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        layers.Conv2D(256, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.4),  # Prevent overfitting
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation="softmax")
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Lower learning rate
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model


In [ ]:
# Image Data Generators for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Rotate images randomly up to 30 degrees
    width_shift_range=0.2,  # Shift image horizontally
    height_shift_range=0.2,  # Shift image vertically
    brightness_range=[0.8, 1.2],  # Adjust brightness
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Zoom-in/out transformations
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load images from DataFrame
train_generator = train_datagen.flow_from_dataframe(
    train_df, x_col="image_path", y_col="label", target_size=(64, 64), batch_size=32, class_mode="sparse"
)

val_generator = val_datagen.flow_from_dataframe(
    val_df, x_col="image_path", y_col="label", target_size=(64, 64), batch_size=32, class_mode="sparse"
)

In [ ]:
model = build_model()

history = model.fit(train_generator, 
                    validation_data=val_generator, 
                    epochs=20, 
                    verbose=1)

In [ ]:
# Plot accuracy and loss curves
def plot_training_curves(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss')

    plt.show()

plot_training_curves(history)

In [ ]:
# Get true labels and predictions
y_true = val_generator.classes
y_pred = np.argmax(model.predict(val_generator), axis=1)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", xticklabels=train_generator.class_indices.keys(), 
            yticklabels=train_generator.class_indices.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print(classification_report(y_true, y_pred, target_names=train_generator.class_indices.keys()))


In [ ]:
# Load your trained Keras model
model = tf.keras.models.load_model("chess_piece_model.keras")

# Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
mobile_model_path = os.path.join(
    '..',
    'mobile_application',
    'assets',
    'chess_piece_model.tflite'
)

with open(mobile_model_path, "wb") as f:
    f.write(tflite_model)
